In [2]:
import os, sys, glob
import pandas as pd

## Load clinical sample data

In [10]:
# load the complete samples' summary table

data_dir = '../data/dataset_joined'
sample_df = pd.read_csv(data_dir + '/_samples.csv')
sample_df.shape

(70, 7)

In [11]:
def find_sample_file(label): 
    ret = glob.glob(data_dir+ '/**/*'+label+'*ReadsPerGene*' ,recursive=True) 
    return ret[0] if ret else False

In [12]:
# match file name for each sample label
sample_df['filename'] = sample_df['sample_label'].apply(find_sample_file)
# add '_1' for single sample
sample_df['sample_label']= sample_df['sample_label'].apply(lambda x: x + '_1' if '_' not in x else x)
# trim sample label
sample_df['sample_label']=  sample_df['sample_label'].apply(lambda x : '_'.join(x.split('_')[0:2]))
sample_df['sample_label'] = 'X_' + sample_df['sample_label']
sample_df.sort_values(by='condition', inplace=True)
sample_df.set_index('sample_label', inplace=True)
sample_df.tail()

,batch,diagnosis,condition,patient,sex,age,filename
sample_label,,,,,,,
X_R40334B_2,1,ctrl,ctrl,8,M,27.51,../data/dataset_joined\R40334B_2_S22_R1_001.fa...
X_R30875B_2,2,ctrl,ctrl,16,M,52.21,../data/dataset_joined\R30875B_2_S8_R1_001.fas...
X_R30875B_1,2,ctrl,ctrl,16,M,52.21,../data/dataset_joined\R30875B_1_S7_R1_001.fas...
X_R21723B_1,2,ctrl,ctrl,14,M,39.52,../data/dataset_joined\R21723B_1_S1_R1_001.fas...
X_40064_1,3,ctrl,ctrl,28,M,51.00,../data/dataset_joined\40064_1_P5_S4_R1_001.fa...


In [13]:
sample_df.drop(columns=['filename']).to_csv(data_dir + '/samples_joined.csv')

In [14]:
#drop all samples from batch 4
sample_df[sample_df.batch != 4].drop(columns=['filename']).to_csv(data_dir + '/samples_original.csv')

## Read STAR results

In [17]:
def starReadCount2df(fname, sample_label): 
    
    df = pd.read_csv(fname, delimiter='\t',  header=None, skiprows=4)
    df.rename(columns={0: 'gene_id', 1: sample_label}, inplace=True)
    df.drop(axis=0, columns=[2, 3],  inplace=True)
    df.set_index('gene_id',  inplace=True)
    return df

In [19]:
numOfSamples=0
for label , r in sample_df.iterrows(): 
    #print(r)
    df = starReadCount2df(r['filename'], label)
    if numOfSamples == 0: 
        count_matrix = df.copy()
    else: 
        count_matrix = count_matrix.merge(df, left_index=True, right_index=True)
    numOfSamples += 1

In [ ]:
count_matrix.to_csv(data_dir + '/countMatrix_raw.csv')

In [ ]:
# remove suffix in transcript_id 
count_matrix.reset_index(inplace=True)
#count_matrix.transcript_id = count_matrix.gene_id.apply(lambda x : x.split('.')[0])
count_matrix.head()

In [ ]:
count_matrix.set_index(keys='gene_id', inplace=True)

## match transcript_id to genes

In [20]:
import sqlite3 as db
ensembldb = db.connect('c:/python/projects/bioinformatics/rnaseq/data/GRCh38\pyensembl\GRCh38\ensembl104\Homo_sapiens.GRCh38.104.gtf.db')

In [ ]:
# build string list to fit into SQL query
trans_id_list= "('"+"', '".join(count_matrix.index) + "')"
trans_id_list[0:100]

In [ ]:
sql_str = "SELECT  gene_id,  gene_name  from gene WHERE gene_id IN {} ".format(trans_id_list)
trans2gene = pd.read_sql_query(sql_str, ensembldb)

trans2gene.head()

In [ ]:
trans2gene.set_index('gene_id', inplace=True)

In [ ]:
df = count_matrix.merge(trans2gene, left_index=True, right_index=True)

In [ ]:
df.to_csv(data_dir + '/countMatrix_with_gene_id.csv')

In [ ]:
df2 = df
df2 = df2[df2['gene_name']!='']
df2.set_index('gene_name', inplace=True)
df2 = df2.groupby('gene_name').sum()
df2.to_csv(data_dir + '/countMatrix_include_gene_name.csv')